# Plan de Implementación – Piloto 90 días

## Semana 0–1: Go-Live inicial con datos sintéticos
- Se despliega la infraestructura dentro del hospital (on-premises): la API, Redis con persistencia (AOF) y su réplica, de modo que los datos sensibles (PHI) nunca se guarden en la nube.

- Se configura el proxy corporativo para que todas las conexiones externas (FHIR, FDA, wearables) pasen por él y no salgan directo a Internet.

- Se simula un flujo real de datos clínicos sin PHI usando mensajes HL7 sintéticos. Estos mensajes se convierten a un formato JSON uniforme/normalizan, se les hace un control de calidad básico (QC) para marcar problema, y solo los errores graves se envían a la DLQ.

- Los datos válidos junto con sus QC, se precalculan y se guardan en la bd Redis en estructuras eficientes. Gracias a esto, la API y la interfaz clínica pueden responder en menos de 100 ms mostrando tendencias, últimos valores y alertas listas para el usuario, ...


## Semanas 2–3: Calidad de datos
Se implementa un **parser robusto de unidades** para interpretar correctamente los valores de laboratorio. Además, se incorpora un mecanismo de **detectar y unificar registros repetidos** (Ids dintintos pero igual información). También se habilitaría una **DLQ con capacidad de replay**, lo que hará que ningún dato se pierda aunque falle el procesamiento inicial.


## Semanas 4–6: Integraciones externas y FDA seguro
Se podrán conectar de forma controlada las **fuentes externas**. Cualquier **fetcher de FHIR** traerá datos pasando siempre por el proxy, un **fetcher de la FDA** comprueba si la información sigue vigente y además define un tiempo de vida (TTL) en caché para evitar reutilizar datos que ya estén caducados. La información obtenida se almacena en una caché local en Redis y se utiliza únicamente para añadir valor a los insights de forma asíncrona, garantizando que la lectura clínica principal no se vea interrumpida. En el front se muestra la fecha y versión de los datos de la FDA, de modo que cualquier desactualización sea evidente. Nno se mostrarán datos antiguos sin advertencia.

## Semanas 7–10: IA asistiva segura / UX 
Se incorpora la **IA asistiva segura**, utilizando un **RAG** basado en documentos internos desidentificados y un **LLM on-premises** configurado en modo asistivo, con **citas obligatorias**, capacidad de **abstenerse de responder cuando falta evidencia**. Esta fase se validaría con un set de preguntas y respuestas clínicas en los principales idiomas que se manejen/, buscando alcanzar una precision alta, al ser algo medico se esperaría que sea por lo menos >90%. Luego, nos enfocariamos en la **experiencia de usuario para pacientes mayores**, desplegando un **modo kiosco** con letra grande, flujos bastante simplificados, acompañado de entrenamiento rápido para cuidadores o enfermeras. La usabilidad se valida con pruebas en 10 pacientes sintéticos, estableciendo como meta una **buena tasa de éxito en la realización de tareas**



## Ultimas semanas 

Se realiza el refuerzo final de seguridad antes de la demo clínica. Se documentan y prueban los manuales que describen cómo operar el sistema en situaciones específicas para operaciones críticas. Se ejecutan pruebas simulando fallos, esto puedes ser como caída del proxy o pérdida de conexión, verificando que la interfaz siga operativa con datos que ha sido cacheados. 

Además, como ultimo recurso a probar, se lleva a cabo alguna simulación controlada de un ataque interno para revisar accesos, y límites de seguridad. Finalmente, se organiza una demo clínica con los doctores oncólogos utilizando casos sintéticos realistas, mostrando métricas en vivo de latencia, disponibilidad y calidad de datos.